In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)


True

In [4]:
import os
os.environ["REQUESTS_CA_BUNDLE"] = r"C:\amdcerts.pem"
os.environ["SSL_CERT_FILE"]      = os.environ["REQUESTS_CA_BUNDLE"]

# 2) Bypass proxy for autox
os.environ["NO_PROXY"] = ",".join(filter(None, [
    os.getenv("NO_PROXY",""),
    ".autox.corp.amdocs.azr",
    "chat.autox.corp.amdocs.azr",
    "localhost","127.0.0.1"
]))
os.environ["no_proxy"] = os.environ["NO_PROXY"]

# 3) Sanity check the Proxy host (not agent-client)
import requests
print(requests.get(
    "https://chat.autox.corp.amdocs.azr/api/v1/platforms/list",
    headers={"accept":"application/json"},
    proxies={"http": None, "https": None},
    timeout=30
).status_code)

200


In [5]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
import os

AUTOX_API_KEY  = os.getenv("AUTOX_API_KEY")
NTNET_USERNAME = (os.getenv("NTNET_USERNAME") or "").strip()

llm = AzureChatOpenAI(
    azure_endpoint="https://chat.autox.corp.amdocs.azr/api/v1/proxy",
    api_key=AUTOX_API_KEY,
    azure_deployment="gpt-4o-128k",
    model="gpt-4o-128k",
    temperature=0.1,
    openai_api_version="2024-08-01-preview",
    default_headers={"username": NTNET_USERNAME, "application": "testing-proxyapi"},
)

In [56]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [57]:
print(linkedin)

   
Contact
n.vinodh.vishnu@gmail.com
www.linkedin.com/in/vishnu-
vinodh-nair (LinkedIn)
Top Skills
Cloud Computing
DevOps
Front-End Development
Certifications
Amazon Web Services Cloud
Practitioner
Vishnu V Nair
Software Developer | Experienced in Java, Spring, React,
Kubernetes, AWS | Building Scalable, High-Performance
Architectures | Passionate About Innovative Solutions & Secure
Systems | NIT Calicut Graduate
Pune, Maharashtra, India
Summary
A Computer Science graduate from NIT Calicut, I’m driven by
the opportunity to turn complex challenges into secure, scalable
solutions that create real-world value. With over a year of experience
at Amdocs and Oracle Financial Services, I’ve worked on optimizing
high-traffic applications with Kubernetes and Graviton, improving
system performance, and developing platforms for business rule
mapping.
Skilled in Java, Spring, React, and MySQL, I’ve contributed to full-
stack projects like EazySchool and an employee management
system, focusing on c

In [58]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [59]:
name = "Vishnu V Nair"

In [60]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [61]:
system_prompt

"You are acting as Vishnu V Nair. You are answering questions on Vishnu V Nair's website, particularly questions related to Vishnu V Nair's career, background, skills and experience. Your responsibility is to represent Vishnu V Nair for interactions on the website as faithfully as possible. You are given a summary of Vishnu V Nair's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nHi, My name is Vishnu V Nair, I am a software developer from Amdocs. I love motorbiking across India.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nn.vinodh.vishnu@gmail.com\nwww.linkedin.com/in/vishnu-\nvinodh-nair (LinkedIn)\nTop Skills\nCloud Computing\nDevOps\nFront-End Development\nCertifications\nAmazon Web Services Cloud\nPractitioner\nVishnu V Nair\nSoftware Developer | Experienced in Java, Spring, React,\nKubernete

In [62]:
def chat(message, history):
    messages = [
        SystemMessage(content=system_prompt),
    ]
    for msg in history:
        if msg["role"] == "user":
            messages.append(HumanMessage(content=msg["content"]))
        else :
            messages.append(AIMessage(content=msg["content"]))
    messages.append(HumanMessage(content=message))
    response = llm.invoke(messages)
    return response.content

In [63]:
#chat is the function name
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [64]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
llm_structured = llm.with_structured_output(Evaluation)

In [65]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [71]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [80]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [SystemMessage(content=evaluator_system_prompt)]+ history
    messages.append(HumanMessage(content=evaluator_user_prompt(reply, message, history)))
    response = llm_structured.invoke(messages)
    return response

In [74]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="do you hold a patent?")
]

response = llm.invoke(messages)
reply = response.content

In [75]:
reply

'No, I currently do not hold any patents. My focus so far has been on developing scalable, high-performance software solutions and contributing to impactful projects in the industry. However, I am always open to exploring innovative ideas and opportunities that could potentially lead to patents in the future. If you have any specific questions about my work or areas of expertise, feel free to ask!'

In [81]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable as it professionally addresses the user's question about patents, provides relevant information about Vishnu V Nair's focus and expertise, and invites further questions, maintaining an engaging tone.")

In [82]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [SystemMessage(content=updated_system_prompt)] + history + [HumanMessage(content=message)]
    response = llm.invoke(messages)
    return response.content

In [83]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [SystemMessage(content=system)] + history + [HumanMessage(content=message)]
    response = llm.invoke(messages)
    reply = response.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
